# Lesson 6 - Creating an Healthcare Provider Agent using LangGraph, MCP, and A2A

In this lesson, you will build a third agent: a Healthcare Provider Agent. This agent demonstrates a powerful combination of technologies:

1.  **[MCP (Model Context Protocol)](https://modelcontextprotocol.io/)**: You will build a server that exposes a tool to find doctors from a JSON file.
2.  **[LangGraph]https://www.langchain.com/langgraph)**: You will build an agent that uses this MCP tool.
3.  **A2A**: You will wrap the LangGraph agent in an A2A server.
4.  **LiteLLM**: You will use LiteLLM to interact with the Google Gemini API.


## 6.1. Create the MCP Server

The MCP server code is provided in `mcpserver.py`. This server exposes a tool called `list_doctors` which queries a local `doctors.json` file.


In [2]:
from IPython.display import Code, display

display(Code("mcpserver.py"))

"""
MCP (Model Context Protocol) Server
-----------------------------------
This file is NOT an A2A agent. It is a tool/database server.
Think of MCP an adapter that securely exposes a specific set of tools (like querying a doctor database)
to any AI agent that knows how to speak the MCP protocol.

Why use this? 
Instead of giving your AI agent raw database credentials, you build an MCP server.
The AI agent asks the MCP server "Can you run this query for me?", and the server does the work.
This is highly secure and modular.
"""

import json
from pathlib import Path

# FastMCP makes it easy to create a tool server
from mcp.server.fastmcp import FastMCP

# 1. Initialize the MCP server
mcp = FastMCP("doctorserver")

# 2. Load the actual data the tools will interact with (our fake doctor database)
# Adjusted path to match project structure
doctors: list = json.loads(Path("data/doctors.json").read_text())


# 3. Define the Tool
# The `@mcp.tool()` decorator tells the MCP server to expose this Python function to AI agents.
# The AI agent can read the docstring (""""This tool returns..."""") to understand what the tool does.
@mcp.tool()
def list_doctors(state: str | None = None, city: str | None = None) -> list[dict]:
    """This tool returns a list of doctors practicing in a specific location. The search is case-insensitive.

    Args:
        state: The two-letter state code (e.g., "CA" for California).
        city: The name of the city or town (e.g., "Boston").

    Returns:
        A JSON string representing a list of doctors matching the criteria.
        If no criteria are provided, an error message is returned.
        Example: '[{"name": "Dr John James", "specialty": "Cardiology", ...}]'
    """
    # Input validation: ensure at least one search term is given.
    if not state and not city:
        return [{"error": "Please provide a state or a city."}]

    target_state = state.strip().lower() if state else None
    target_city = city.strip().lower() if city else None

    return [
        doc
        for doc in doctors
        if (not target_state or doc["address"]["state"].lower() == target_state)
        and (not target_city or doc["address"]["city"].lower() == target_city)
    ]


# 4. Kick off server if file is run
# "stdio" (Standard Input/Output) means the AI agent will communicate with this server 
# using the terminal (running it as a background process), rather than over a web port.
if __name__ == "__main__":
    mcp.run(transport="stdio")

## 6.2. Test with LangGraph and MCP Client

Now you will verify that you can connect to the MCP server and use it within a LangChain/LangGraph agent. You use `MultiServerMCPClient` to connect to the local script via `stdio` transport.


In [3]:
import asyncio

from IPython.display import Markdown, display
from langchain.agents import create_agent
from langchain_litellm import ChatLiteLLM
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.sessions import StdioConnection

from helpers import setup_env

setup_env()

In [4]:
mcp_client = MultiServerMCPClient(
    {
        "find_healthcare_providers": StdioConnection(
            transport="stdio",
            command="uv",
            args=["run", "mcpserver.py"],
        )
    }
)

In [5]:
agent = create_agent(
    ChatLiteLLM(
        model="gemini/gemini-3-flash-preview",
        # For Vertex AI:
        # model="vertex_ai/gemini-3-flash-preview",
        max_tokens=1000,
    ),
    asyncio.run(mcp_client.get_tools()),
    name="HealthcareProviderAgent",
    system_prompt="Your task is to find and list healthcare providers using the find_healthcare_providers MCP Tool based on the users query.",
)

In [6]:
prompt = "I'm based in Austin, TX. Are there any Psychiatrists near me?"
response = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ]
    }
)

In [7]:
display(Markdown(response["messages"][-1].content))

Yes, there is a psychiatrist located in Austin, TX:

**Dr. Jessica Coffey**
*   **Specialty:** Psychiatry
*   **Address:** 1201 West 38th Street, Suite 210, Austin, TX 78705
*   **Phone:** (512) 555-0199
*   **Email:** j.coffey@austinmindhealth.com
*   **Experience:** 13 years
*   **Board Certified:** Yes
*   **Accepting New Patients:** Yes
*   **Insurance Accepted:** Blue Cross Blue Shield, UnitedHealth, Aetna, Humana
*   **Hospital Affiliations:** Ascension Seton Medical Center, St. David's Medical Center

## 6.3. Wrap in A2A Server

We have encapsulated this logic in `a2a_provider_agent.py`. This file handles the MCP client connection and agent initialization and wraps the LangGraph agent as an A2A server using the [`langgraph-a2a-server`](https://github.com/5enxia/langgraph-a2a-server/) library.

In [ ]:
from IPython.display import Code, display

display(Code("a2a_provider_agent.py"))

## 6.4. Run the Provider Server

Activate the Provider Agent in Terminal 3.
- Open a terminal as instructed below.
- Type `uv run a2a_provider_agent.py`

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open a new terminal window in your Jupyter environment to run the server.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 6.7. Resources

- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [LangChain MCP Adapters](https://docs.langchain.com/oss/python/langchain/mcp)
- [LiteLLM](https://docs.litellm.ai/)


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>